In [1]:
from Tools_Libraries import *
from IonModel_Libraries import *
from LimitSetting_Libraries import *
from WIMP_Libraries import *
from DP_Libraries import *
from Ultimate_Libraries import *

In [4]:
Exp = Experiment()
Exp.Voltage = 100
Exp.SigmaeV = 20
Exp.Exposure = 365
Exp.Fano = 0.2
Exp.TriggerNsigma = 5

L = LimitSetter(CL=90,Nvalues=1000)

Energyscale = "eVee"
EminAnalysis = 0 #
EmaxAnalysis = 30
nptx = 1000
OR = Oracle(EminAnalysis,EmaxAnalysis,nptx,L,"DP")

def CalculateLimit(Exp = Experiment(),OR = Oracle(), Energyscale="eV", signaltype="DP",mass=10,massindex=False,bash=False):
    
    assert OR._SignalType in ["DP","WIMP"]
    assert Energyscale in ["eV","eVee"]
    
    # get analysis range to determine the  energy range for the plot
    Emin, Emax = OR.GetAnalysisRange()
    nptx = OR.GetAnalysisNptx()
    logx = False
    x = np.linspace(Emin,Emax,nptx) if logx==False else np.logspace(np.log10(Emin),np.log10(Emax),nptx)
    
    # now that we have extracted the energy range for the plot, we can set the analysis threshold at NTriggerSigma * Sigma
    Emin = Exp.TriggerNsigma*Exp.SigmaeV if (Energyscale=="eV") else Exp.TriggerNsigma*Exp.SigmaeVee
    OR.SetAnalysisRange((Emin,Emax))

    
    #if(Exp.TriggerNsigma==999):
    #    Exp.ImportEfficiencyCurve("/sps/edelweis/jupytershare/eff_ub13a000.txt")
    #    Exp.TriggerNsigma = 0
        
    Compton = Spectrum("ER",Exp)
    Compton.SetFunc(lambda x: np.ones_like(x)*1e-2,0,1e9)
    ComptonEnergySpectrum = Compton.GetEphononSmearedFunc(Energyscale)
    
    HeatOnly = Spectrum("HO",Exp)
    HeatOnly.SetFunc(fonctionHOdivided1000,0,1e9)
    HeatOnlyEnergySpectrum = HeatOnly.GetEphononSmearedFunc(Energyscale)
       
    if OR._SignalType=="DP":
        D = DP_Parameters()
        if massindex==False:
            D.SetDPmass(mass) # eV/c2
        else:
            D.SetDPmassindex(mass) # eV/c2
        actualmass = D.GetDPmass()
        D.SetKappa(1e-14)
        DarkPhotons = Spectrum(D,Exp)
        DarkPhotonsEnergySpectrum = DarkPhotons.GetEphononSmearedLine(Energyscale)
        
    elif OR._SignalType == "WIMP":
        W = WIMP_Parameters()
        W.WIMPmass = mass
        actualmass = W.WIMPmass
        W.CrossSection = 1e-4
        Wimps = Spectrum(W,Exp)
        WimpsEnergySpectrum = Wimps.GetEphononSmearedFunc(Energyscale)
    else:
        print("ERROR")
    

    OR.AddBackground(ComptonEnergySpectrum,"Compton Bkg")
    OR.AddBackground(HeatOnlyEnergySpectrum,"HO Bkg")
  
    if OR._SignalType=="DP":
        OR.AddSignal(DarkPhotonsEnergySpectrum)
        OR.SetVariableOfInterest(D.GetKappa()**2)
    elif OR._SignalType == "WIMP":
        OR.AddSignal(WimpsEnergySpectrum)
        OR.SetVariableOfInterest(W.CrossSection)
    
    fb = OR.GetTotalBackground()
    fs = OR.GetSignal()
    print()
    ROIlist = OR.CalculateROI_Bounds(fs,fb)
    #print("ROIlist=",ROIlist)
    if OR._SignalType=="DP":
        Dexcluded = copy.copy(D)
        Dexcluded.SetKappa(kappaexcluded)
        DarkPhotonsexcluded = Spectrum(Dexcluded,Exp)
        #print("Kappa excluded {:.2e}".format(kappaexcluded))
    elif OR._SignalType == "WIMP":
        sigmaexcluded = OR.CalculateExcludedVariableOfInterest()
        Wexcluded = copy.copy(W)
        Wexcluded.CrossSection = sigmaexcluded
        #print("sigmaexcluded = ",sigmaexcluded)
        Wimpsexcluded = Spectrum(Wexcluded,Exp)
        #print("cross section excluded {:.2e} cm2".format(sigmaexcluded))
        
    muexcluded = OR.GetExcludedMu()
    #print("mu excluded = {:.2e}".format(muexcluded))
      
    if (bash == False):
        print("bash is here :",bash)
        fig, ax = plt.subplots(figsize=(16,9))
        
        for name in OR.GetBackgroundNames():
            tf1 = OR.GetBackground(name)
            plt.plot(x,tf1(x),label=name,lw=4)
            
        yb = fb(x)
        ax.plot(x,yb,color="black",ls='--',label="Total Bkg",lw=4)
        #plt.locator_params("x",nbins=15)
        
        ax.set_ylim(bottom=1e-4,top=1e4)
        ax.set_yscale('log')
        if logx:
            ax.set_xscale('log')
        
        if OR._SignalType=="DP":
            title1 = "DP signal excluded, m={:.2f} eV/c$^2$".format(actualmass)
            title2 = "$\kappa={:.2e}$, $\mu={:.2f}$".format(kappaexcluded,muexcluded)
            fsexcluded = DarkPhotonsexcluded.GetEphononSmearedLine(Energyscale)
        elif OR._SignalType == "WIMP":
            title1 = "WIMP signal excluded, m={:.2f} GeV/c$^2$".format(actualmass)
            title2 = "$\sigma$={:.2e} pb, $\mu={:.2f}$".format(sigmaexcluded,muexcluded) 
            fsexcluded = Wimpsexcluded.GetEphononSmearedFunc(Energyscale)
            
        yexcl = fsexcluded(x)
        ax.plot(x,yexcl,label=title1,lw=4,color='red')
        ax.fill_between(x,yb,yexcl, alpha=0.5,where=(yexcl>yb),color='red')
        ax.plot([], [],' ',label=title2)

        
        if(Energyscale=="eVee"):
            ftransform = lambda x : x/Exp.Epsilon
            finv = lambda x : x*Exp.Epsilon
            secax = ax.secondary_xaxis('top', functions=(ftransform,finv))
        else:
            ftransform = lambda x : x/(1+Exp.Voltage/Exp.Epsilon)/Exp.Epsilon
            finv = lambda x : x*(1+Exp.Voltage/Exp.Epsilon)/Exp.Epsilon
            secax = ax.secondary_xaxis('top', functions=(ftransform,finv),color="royalblue")
         
        #ax.locator_params(tight=True, nbins=4)
        secax.set_xlabel('Electron number of e$^-$/ h$^+$ for electron recoils')
        ax.grid(which='both')
        ax.set_xlabel("Energy [{}]".format(Energyscale))
        ax.set_ylabel("Events / {} kg.days / {}".format(Exp.Exposure,Energyscale))
        #plt.gca().yaxis.set_major_locator(ticker.LogLocator(base=10.0, numticks=15))
        OR.PlotROIs(color="red",ls="--",prop=1)
        OR.PlotThreshold()
        ax.legend()
    print("done")
    if OR._SignalType=="DP":
        return actualmass, kappaexcluded, muexcluded 
    elif OR._SignalType == "WIMP":
        return actualmass, sigmaexcluded, muexcluded 
        
CalculateLimit(Exp = Exp,OR = OR,Energyscale = Energyscale , signaltype="DP", mass=10,massindex=False,bash=True)        



Existing PoissonDataFile with 1000 precalculated values is used
Existing PoissonDataFile with 1000 precalculated values is used

final ROI list consists of 1 ROI(s)
[11.70,12.29] eV


NameError: name 'kappaexcluded' is not defined

In [ ]:
def launchLimits():
    save = False
    signaltype="DP"
    massmin,massmax,nmasses = 0.07,10,5 
    logmasses = True
    masses = np.logspace(np.log10(massmin),np.log10(massmax),nmasses) if (logmasses==True) else np.linspace(massmin,massmax,nmasses)
   
   
    vectruemass, vecvarexcl, vecmuexcl = [], [], []
    for m in masses:
        truemass,varexcl,muexcl = CalculateLimit(Exp = Exp,OR = OR,Energyscale = Energyscale , signaltype=signaltype, mass=m,massindex=False,bash=True) 
        vectruemass.append(truemass)
        vecvarexcl.append(varexcl)
        vecmuexcl.append(muexcl)
    
    if save:
        namefile = "toto-{}-{}kg.days.txt".format(signaltype,Exposure)
        with open(namefile,'w') as f:
            [f.write("{:.2f} {:.4e} {:.2f}\n".format(vectruemass[i],vecvarexcl[i],vecmuexcl[i])) for i,m in enumerate(vectruemass)]
        
    fig, ax = plt.subplots()
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_xlabel("Mass [eV/c$^2$]") if signaltype=="DP" else plt.xlabel("Mass [GeV/c$^2$]")
    ax.set_ylabel("$\kappa$") if signaltype=="DP" else plt.ylabel("$\sigma$ [pb]") 
    ax.grid(which='both') 
    ax.plot(vectruemass,vecvarexcl,'o--')
    plt.show()

#def savevalues(namefile)
launchLimits()


    
#